# <center> Lista 5

### Objetivo: realizar a predição para obter o preço de venda de um imóvel a partir do seguinte dataset "Boston House Price Dataset". 

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from ml_libs import Regressao_linear_gd
from ml_libs import metricas
from importlib import reload

In [26]:
reload(Regressao_linear_gd)
reload(metricas)

<module 'ml_libs.metricas' from '/home/michel/Documentos/Machine-Learning/Lista 5/ml_libs/metricas.py'>

In [3]:
cols = ['CRIM ','ZN','INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX','PTRATIO','B','LSTAT','MEDV']
df = pd.read_csv('housing.csv',names=cols, sep='\s+')

In [4]:
df.head(5)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


### Informações sobre Atributos:

    01. CRIM    Taxa de criminalidade per capita por cidade
    02. ZN      proporção de terrenos residenciais zoneada para lotes mais 25.000 pés quadrados
    03. INDUS   proporção de hectares comerciais não varejistas por cidade
    04. CHAS    Variável dummy Charles River (= 1 se limites do setor rio; 0 caso contrário)
    05. NOX     Concentração de óxidos nítricos (partes por 10 milhões)
    06. RM      média de quartos por habitação
    07. AGE     Proporção de unidades ocupadas pelo proprietário construídas antes de 1940
    08. DIS     Distâncias ponderadas pelo para cinco centros de emprego em Boston
    09. RAD     Índice de acessibilidade a rodovias radiais
    10. TAX     TAXA DE IMPOSTO DE IMPOSIÇÃO DE VALOR TOTAL POR 10.000
    11. PTRATIO Proporção aluno-professor por cidade
    12. B       1000(Bk - 0,63)^2 onde Bk é a proporção de negros pela cidade
    13. LSTAT   % Status menor da população
    14. MEDV    Valor mediano de residências ocupadas pelo proprietário em US$1000

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
CRIM       506 non-null float64
ZN         506 non-null float64
INDUS      506 non-null float64
CHAS       506 non-null int64
NOX        506 non-null float64
RM         506 non-null float64
AGE        506 non-null float64
DIS        506 non-null float64
RAD        506 non-null int64
TAX        506 non-null float64
PTRATIO    506 non-null float64
B          506 non-null float64
LSTAT      506 non-null float64
MEDV       506 non-null float64
dtypes: float64(12), int64(2)
memory usage: 55.4 KB


In [6]:
df.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677082,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


In [7]:
df.hist(bins=20, figsize=(15,15));

In [8]:
df.corr().style.set_precision(4)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
CRIM,1,-0.2005,0.4066,-0.05589,0.421,-0.2192,0.3527,-0.3797,0.6255,0.5828,0.2899,-0.3851,0.4556,-0.3883
ZN,-0.2005,1,-0.5338,-0.0427,-0.5166,0.312,-0.5695,0.6644,-0.3119,-0.3146,-0.3917,0.1755,-0.413,0.3604
INDUS,0.4066,-0.5338,1,0.06294,0.7637,-0.3917,0.6448,-0.708,0.5951,0.7208,0.3832,-0.357,0.6038,-0.4837
CHAS,-0.05589,-0.0427,0.06294,1,0.0912,0.09125,0.08652,-0.09918,-0.007368,-0.03559,-0.1215,0.04879,-0.05393,0.1753
NOX,0.421,-0.5166,0.7637,0.0912,1,-0.3022,0.7315,-0.7692,0.6114,0.668,0.1889,-0.3801,0.5909,-0.4273
RM,-0.2192,0.312,-0.3917,0.09125,-0.3022,1,-0.2403,0.2052,-0.2098,-0.292,-0.3555,0.1281,-0.6138,0.6954
AGE,0.3527,-0.5695,0.6448,0.08652,0.7315,-0.2403,1,-0.7479,0.456,0.5065,0.2615,-0.2735,0.6023,-0.377
DIS,-0.3797,0.6644,-0.708,-0.09918,-0.7692,0.2052,-0.7479,1,-0.4946,-0.5344,-0.2325,0.2915,-0.497,0.2499
RAD,0.6255,-0.3119,0.5951,-0.007368,0.6114,-0.2098,0.456,-0.4946,1,0.9102,0.4647,-0.4444,0.4887,-0.3816
TAX,0.5828,-0.3146,0.7208,-0.03559,0.668,-0.292,0.5065,-0.5344,0.9102,1,0.4609,-0.4418,0.544,-0.4685


### Selecionando as features X e label y

In [17]:
X = df.values[:, :-1]
y = df.values[:, -1]

### Randomização dos dados usando semente (seed)

In [10]:
rs = np.random.RandomState(1)
shuffled_indices = rs.permutation(X.shape[0])
shuffled_indices

array([307, 343,  47,  67, 362, 132, 292,  31, 218,  90, 481, 344, 119,
        66, 312, 407, 376, 225, 201, 147, 315, 146, 314, 324,  65, 425,
        34, 345, 261, 389, 351, 245, 204, 447, 330, 494, 446,   4, 406,
       486, 329, 334, 223, 475, 478, 419, 162, 394, 165, 102, 473, 242,
         5, 168, 490,  62, 399, 311,  17, 233, 420,  85, 397, 430, 214,
       179, 232, 342, 294, 164, 326, 386, 366,  29, 107, 483, 370, 285,
       159, 291, 128, 117, 189, 435, 350, 298, 449, 325, 497, 135, 503,
       257, 491,   6, 427,  11, 501,  92, 224, 110, 426, 443,  42,  58,
       385,  78, 424, 160, 185, 101, 268, 173, 428, 306, 258, 192, 339,
        18,  40, 267, 328, 454,  41, 361, 289, 498, 293, 502, 172,  80,
        46, 318,  93, 392,  81, 304, 142, 456, 180, 380, 355, 139, 106,
       238, 290, 246, 320, 467,  95, 122, 260,  23,  13,  61, 377,  39,
       365, 272, 208, 236,  98, 364, 249, 421, 422, 286, 347, 378, 127,
        82, 463, 207,  27,  89, 469,  73, 187, 171, 161,  69, 22

In [11]:
X = X[shuffled_indices]
y = y[shuffled_indices]

In [18]:
rs.shuffle(X)
rs.shuffle(y)

In [19]:
X[:1]

array([[  8.20058,   0.     ,  18.1    ,   0.     ,   0.713  ,   5.936  ,
         80.3    ,   2.7792 ,  24.     , 666.     ,  20.2    ,   3.5    ,
         16.94   ]])

In [20]:
y[:1]

array([23.4])

### Selecionando 70% dos dados para treino e 30% para teste

In [31]:
n = int(np.count_nonzero(y) * 0.70)
X_train = X[:n]
y_train = y[:n]
X_test = X[n:]
y_test = y[n:]

In [27]:
model = Regressao_linear_gd.RegressaoLinearSimples();

In [34]:
model.fit(X_train,y_train);

ValueError: operands could not be broadcast together with shapes (354,13) (354,) 